In [20]:
import pandas as pd
base_info = pd.read_csv('./data/train/base_info.csv')

In [24]:
base_info['orgid'].nunique()
base_info['jobid'].max()

400000000000776817

In [3]:
nums, shapes = base_info.shape
# 1. 删掉缺失值较多的特征
for name, cout in base_info.isnull().sum().items():
    if cout * 1.0 / nums >= 0.7:
        print(name, cout)
        del base_info[name]
        
# 删除类别不同较多的列
for name, cout in base_info.nunique().items():
    if (cout == 1 or cout * 1.0 / nums >= 0.8) and name != 'id':
        print(name, cout)
        del base_info[name]

print(base_info.info())

parnum 22526
exenum 23487
ptbusscope 24865
enttypeminu 17595
midpreindcode 24865
protype 24831
reccap 17781
forreccap 24638
forregcap 24615
congro 24616
dom 23278
opscope 20815
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24865 entries, 0 to 24864
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             24865 non-null  object 
 1   oplocdistrict  24865 non-null  int64  
 2   industryphy    24865 non-null  object 
 3   industryco     24864 non-null  float64
 4   enttype        24865 non-null  int64  
 5   enttypeitem    16651 non-null  float64
 6   opfrom         24865 non-null  object 
 7   opto           8825 non-null   object 
 8   state          24865 non-null  int64  
 9   orgid          24865 non-null  int64  
 10  jobid          24865 non-null  int64  
 11  adbusign       24865 non-null  int64  
 12  townsign       24865 non-null  int64  
 13  regtype        24865 non-null  int64  
 14  empnu

In [4]:
import numpy as np
from sklearn.impute import KNNImputer

def knn_fill_nan(X):
    imputer = KNNImputer(n_neighbors=5, weights="uniform")
    return imputer.fit_transform(X)

d:企业唯一标识, oplocdistrict:行政区划代码, industryphy:行业类别代码,  :行业细类代码, dom:经营地址, opscope:经营范围, enttype:企业类型, enttypeitem:企业类型小类, opfrom:经营期限起, opto:经营期限止, state:状态, orgid:机构标识, jobid:职位标识, adbusign:是否广告经营, townsign:是否城镇, regtype:主题登记类型, empnum:从业人数, compform:组织形式, parnum:合伙人数, exenum:执行人数, opform:经营方式, ptbusscope:兼营范围, venind:风险行业, enttypeminu:企业类型细类, midpreindcode:中西部优势产业代码, protype:项目类型, oploc:经营场所, regcap:注册资本（金）, reccap:实缴资本, forreccap:实缴资本（外方）, forregcap:注册资本（外方）, congro:投资总额, enttypegb:企业（机构）类型

oplocdistrict industryphy industryphy enttype enttypeitem adbusign adbusign regtype compform enttypegb

使用这些类别特征来对数值型特征进行缺失值处理

In [5]:
category_cols = ['oplocdistrict', 'industryphy', 'industryphy', 'enttype', 'enttypeitem', 'adbusign', 'adbusign', 'regtype','compform' ,'enttypegb','opform']


In [6]:
# 使用 industryphy 类别中数填充 industryco
print(base_info['industryco'].isnull().sum())
grouped = base_info[['industryco','industryphy']].groupby('industryphy')
for name, group in grouped:
    median = group.median()
    base_info.loc[base_info['industryphy'] == name,'industryco'] = base_info[base_info['industryphy'] == name]['industryco'].fillna(median.values[0])
print(base_info['industryco'].isnull().sum())


1
0


In [7]:
# 使用 enttype 类别填充 enttypeitem
print(base_info['enttypeitem'].isnull().sum())
grouped = base_info[['enttype','enttypeitem']].groupby('enttype')
for name, group in grouped:
    base_info.loc[base_info['enttype'] == name,'enttypeitem'] = base_info[base_info['enttype'] == name]['enttypeitem'].fillna(name)
print(base_info['enttypeitem'].isnull().sum())

8214
0


In [8]:
# 使用 enttype 类别填充 enttypeitem
print(base_info['venind'].isnull().sum())
grouped = base_info[['industryphy','venind']].groupby('industryphy')
for name, group in grouped:
    print(name,group.mean())
print(base_info['venind'].isnull().sum())

16428
A venind   NaN
dtype: float64
B venind   NaN
dtype: float64
C venind    2.0
dtype: float64
D venind   NaN
dtype: float64
E venind    3.0
dtype: float64
F venind    3.0
dtype: float64
G venind    2.0
dtype: float64
H venind   NaN
dtype: float64
I venind   NaN
dtype: float64
J venind   NaN
dtype: float64
K venind    3.0
dtype: float64
L venind    2.6875
dtype: float64
M venind    2.888889
dtype: float64
N venind    2.870968
dtype: float64
O venind    2.961476
dtype: float64
P venind    2.961905
dtype: float64
Q venind    2.834711
dtype: float64
R venind    2.827338
dtype: float64
S venind   NaN
dtype: float64
T venind   NaN
dtype: float64
16428


In [9]:
print(base_info[category_cols].isnull().sum())
print(base_info[category_cols].nunique())
print(base_info['venind'].nunique())

oplocdistrict        0
industryphy          0
industryphy          0
enttype              0
enttypeitem          0
adbusign             0
adbusign             0
regtype              0
compform         14234
enttypegb            0
opform           15865
dtype: int64
oplocdistrict    16
industryphy      20
industryphy      20
enttype          17
enttypeitem      36
adbusign          2
adbusign          2
regtype           3
compform          2
enttypegb        53
opform           33
dtype: int64
3


In [10]:
# opform  compform 缺失值多 如何处理呢?
base_info['opform'] = base_info['opform'].replace('01', '01-以个人财产出资').replace('02', '02-以家庭共有财产作为个人出资')
# 暂时删掉试一下啊
del base_info['opform']
del base_info['compform']

In [11]:
# 对数值类进行缺失处理, 采样聚类方法进行处理
category_cols = ['oplocdistrict', 'industryphy', 'industryphy', 'enttype', 'enttypeitem', 'adbusign', 'adbusign', 'regtype','enttypegb']
print(base_info[category_cols].dtypes)
for col in category_cols:
    # 对类别信息采取 one-hot 编码
    category = pd.get_dummies(base_info[col],prefix=col,drop_first=True) 
    base_info[category.columns] = category

oplocdistrict      int64
industryphy       object
industryphy       object
enttype            int64
enttypeitem      float64
adbusign           int64
adbusign           int64
regtype            int64
enttypegb          int64
dtype: object


In [12]:
base_info.dtypes

id                 object
oplocdistrict       int64
industryphy        object
industryco        float64
enttype             int64
                   ...   
enttypegb_6150      uint8
enttypegb_6180      uint8
enttypegb_6810      uint8
enttypegb_9100      uint8
enttypegb_9600      uint8
Length: 159, dtype: object

In [13]:
base_info.drop(category_cols,axis=1,inplace=True)
print(base_info.dtypes)

id                 object
industryco        float64
opfrom             object
opto               object
state               int64
                   ...   
enttypegb_6150      uint8
enttypegb_6180      uint8
enttypegb_6810      uint8
enttypegb_9100      uint8
enttypegb_9600      uint8
Length: 152, dtype: object


In [14]:
# 时间转换, 暂时先抽取年份特征

base_info['opfrom'] = pd.to_datetime(base_info.opfrom)
base_info['opfrom_year'] = base_info['opfrom'].dt.year.astype('int')

base_info['opto'] = pd.to_datetime(base_info.opto)
base_info['opto_year'] = base_info['opto'].dt.year.fillna(-1).astype('int')

del base_info['opfrom']
del base_info['opto']

In [15]:
base_info.select_dtypes('object')
del base_info['oploc']



In [16]:
base_info.isnull().sum().sort_values(ascending=False)

venind            16428
empnum             5250
regcap              191
opto_year             0
enttype_3200          0
                  ...  
enttypegb_1123        0
enttypegb_1122        0
enttypegb_1121        0
regtype_4             0
id                    0
Length: 151, dtype: int64

In [17]:
base_info.describe()

,industryco,state,orgid,jobid,townsign,empnum,venind,regcap,oplocdistrict_340200,oplocdistrict_340201,...,enttypegb_5810,enttypegb_6110,enttypegb_6120,enttypegb_6150,enttypegb_6180,enttypegb_6810,enttypegb_9100,enttypegb_9600,opfrom_year,opto_year
count,24865.000000,24865.000000,2.486500e+04,2.486500e+04,24865.000000,19615.000000,8437.000000,2.467400e+04,24865.000000,24865.000000,...,24865.000000,24865.000000,24865.000000,24865.000000,24865.000000,24865.000000,24865.000000,24865.000000,24865.000000,24865.000000
mean,7885.246169,6.164569,3.416682e+17,3.548101e+17,0.578444,4.397961,2.946545,5.151437e+03,0.138428,0.001568,...,0.000121,0.000523,0.000040,0.000161,0.000040,0.000080,0.000925,0.329499,2016.677177,727.344460
std,667.038309,0.582226,1.107349e+16,3.052367e+16,0.493818,15.389206,0.284933,6.777086e+04,0.345355,0.039574,...,0.010984,0.022860,0.006342,0.012683,0.006342,0.008968,0.030400,0.470041,3.553619,982.010718
min,0.000000,4.000000,1.410800e+17,1.410800e+17,0.000000,0.000000,1.000000,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1979.000000,-1.000000
25%,7512.000000,6.000000,3.402000e+17,3.402000e+17,0.000000,2.000000,3.000000,1.500000e+01,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2016.000000,-1.000000
50%,8040.000000,6.000000,3.402020e+17,3.402000e+17,1.000000,3.000000,3.000000,8.000000e+01,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2018.000000,-1.000000
75%,8199.000000,6.000000,3.402070e+17,4.000000e+17,1.000000,5.000000,3.000000,5.000000e+02,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2019.000000,2039.000000
max,9700.000000,20.000000,4.000000e+17,4.000000e+17,1.000000,1500.000000,3.000000,5.000100e+06,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2020.000000,2118.000000


In [18]:
base_info['enttypeminu'].isnull().sum()

KeyError: 'enttypeminu'